In [ ]:
%%javascript


# d3SPARQL


This notebook explores using d3SPARQL for visualization

In [ ]:
%%clojure
(refer 'sparqler.core)
(import java.net.URI
           [java.net URLEncoder URLDecoder])
kabob

In [ ]:
%%javascript
function loadCSS(filename){ 

       var file = document.createElement("link");
       file.setAttribute("rel", "stylesheet");
       file.setAttribute("type", "text/css");
       file.setAttribute("href", filename);
       document.head.appendChild(file);

}

//just call a function to load a new CSS:
loadCSS("http://cdn.jsdelivr.net/yasqe/2.11.6/yasqe.min.css");
loadCSS("http://yasr.yasgui.org/dist/yasr.min.css");
loadCSS("http://yasr.yasgui.org/doc/doc.min.css");

## Examples

### ForceGraph

In [ ]:
%%sql
select ?s ?p ?o 
WHERE
{
{
  BIND ("Morphine" as ?c) . 
  ?s ?p1 ?c .
  ?s ?p ?o
}
UNION
{
  BIND ("Morphine" as ?c) . 
  ?o ?p1 ?c .
  ?s ?p ?o
}
}

In [ ]:
%%javascript
beakerx.query = beakerx.getCodeCells('query')[0].code

In [ ]:
%%clojure
(def q (beakerx :get "query"))
(def r (kabob-query q))
(beakerx :set "resultmap" (clojure.set/rename r {:S "s", :P "p", :O "o"}))

In [ ]:
%%clojure
(def qexec (doto (QueryExecutionFactory/sparqlService (get kabob :endpoint) q) (.setBasicAuthentication (get kabob :user) (char-array (get kabob :pass)))))
qexec

In [ ]:
%%clojure
(def results (.execSelect qexec))
(def outputStream (ByteArrayOutputStream.))
(ResultSetFormatter/outputAsJSON outputStream results)
(def json (String. (.toByteArray outputStream)))
(beakerx :set "resultjson" json)

In [ ]:
%%html
<div id="query" style="margin: 10px">
      <h1>d3forcegraph</h1>
      <form class="form-inline">
        <label>SPARQL endpoint:</label>
        <div class="input-append">
          <input id="endpoint" class="span5" value="http://dbpedia.org/sparql" type="text">
          <button class="btn" type="button" onclick="exec()">Query</button>
          <button class="btn" type="button" onclick="exec_offline()">Use cache</button>
        </div>
      </form>
      <textarea id="sparql" class="span9" rows=15 style="width:700px;">
# https://en.wikipedia.org/wiki/History_of_programming_languages
# https://en.wikipedia.org/wiki/Perl
# http://dbpedia.org/page/Perl
# http://dbpedia.org/sparql

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX dbpedia-owl: <http://dbpedia.org/ontology/>
PREFIX dbpprop: <http://dbpedia.org/property/>
PREFIX dbpedia: <http://dbpedia.org/resource/>

SELECT DISTINCT ?lang1 ?lang2 ?lang1label ?lang2label ?lang1value ?lang2value ?lang1year ?lang2year
WHERE {
  ?lang1 rdf:type dbpedia-owl:ProgrammingLanguage ;
         rdfs:label ?lang1name ;
         dbpprop:year ?lang1year .
  ?lang2 rdf:type dbpedia-owl:ProgrammingLanguage ;
         rdfs:label ?lang2name ;
         dbpprop:year ?lang2year .
  ?lang1 dbpedia-owl:influenced ?lang2 .
  FILTER (?lang1 != ?lang2)
  FILTER (LANG(?lang1name) = 'en')
  FILTER (LANG(?lang2name) = 'en')
  BIND (replace(?lang1name, " .programming language.", "") AS ?lang1label)
  BIND (replace(?lang2name, " .programming language.", "") AS ?lang2label)
  FILTER (?lang1year > 1950 AND ?lang1year < 2020)
  FILTER (?lang2year > 1950 AND ?lang2year < 2020)
  # To render older language larger than newer
  BIND ((2020 - ?lang1year) AS ?lang1value)
  BIND ((2020 - ?lang2year) AS ?lang2value)
}
      </textarea>
    </div>
    <div id="result"></div>

In [ ]:
%%javascript
document.getElementById("result").innerHTML = "";

    function exec() {
      console.log("exec")
      var endpoint = d3.select("#endpoint").property("value")
      var sparql = d3.select("#sparql").property("value")
      d3sparql.query(endpoint, sparql, render)
    }
    function render(json) {
      var config = {
        "key1":   "s",       // SPARQL variable name for node1 (optional; default is the 1st variable)
        "key2":   "o",       // SPARQL variable name for node2 (optional; default is the 2nd varibale)
        "label1": "s",  // SPARQL variable name for the label of node1 (optional; default is the 3rd variable)
        "label2": "o",  // SPARQL variable name for the label of node2 (optional; default is the 4th variable)
        "edge1" : "p",
        "radius": 10,
        "charge": -500,
        "distance": 50,
        "width": 1000,
        "height": 750,
        "selector": "#result"
      }
      d3sparql.forcegraph(json, config)
      beakerx.json = json
    }

    function exec_offline() {
      d3.json("vendor/proglang_pair.json", render)
    }
    function toggle() {
      d3sparql.toggle()
    }

render(JSON.parse(beakerx.resultjson))